### Import Libraries

In [1]:
import os
import json
import psycopg2
from typing import List, Dict, Tuple, Optional
import re

print("✅ All libraries imported successfully!")
print(f"Python environment ready for ChatbotV2")

✅ All libraries imported successfully!
Python environment ready for ChatbotV2


### Load Environment Variables and Setup

In [ ]:
from dotenv import load_dotenv
import requests

# Load environment variables from .env file
load_dotenv()

# Get Ollama configuration from .env
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL")

# Get Database configuration from .env
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "database": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD", "")  # Empty string if no password
}

# File paths
METADATA_V2_PATH = "/Users/abdullah/Desktop/ecommerce_RAG_system/data/v2/metadata_v2.json"
TRAINING_SET_PATH = "/Users/abdullah/Desktop/ecommerce_RAG_system/data/v2/training_set.json"

print("="*80)
print("🚀 E-COMMERCE TEXT-TO-SQL CHATBOT V2")
print("="*80)

# Test database connection
print("\n📊 Testing database connection...")
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM orders;")
    order_count = cursor.fetchone()[0]
    cursor.close()
    conn.close()
    print(f"✅ Database connected successfully!")
    print(f"   Orders in database: {order_count:,}")
except Exception as e:
    print(f"❌ Database connection failed: {e}")

# Test Ollama connection
print("\n🤖 Testing Ollama connection...")
try:
    response = requests.post(
        f"{OLLAMA_BASE_URL}/api/generate",
        json={
            "model": OLLAMA_MODEL,
            "prompt": "Say 'Hello'",
            "stream": False,
            "keep_alive": "10m"  # ← Keep in memory for 10 minutes (faster testing)
        },
        timeout=60
    )
    if response.status_code == 200:
        print(f"✅ Ollama connected successfully!")
        print(f"   Model: {OLLAMA_MODEL}")
        print(f"   Strategy: Keep in memory for 10 minutes")
        print(f"   (Auto-unloads after 10 min of inactivity)")
    else:
        print(f"❌ Ollama connection failed: Status {response.status_code}")
except Exception as e:
    print(f"❌ Ollama connection failed: {e}")

print("\n" + "="*80)
print("✅ Setup complete - ready to build ChatbotV2!")
print("="*80)

🚀 E-COMMERCE TEXT-TO-SQL CHATBOT V2

📊 Testing database connection...
✅ Database connected successfully!
   Orders in database: 99,441

🤖 Testing Ollama connection...
✅ Ollama connected successfully!
   Model: qwen2.5-coder:14b
   Strategy: Keep in memory for 10 minutes
   (Auto-unloads after 10 min of inactivity)

✅ Setup complete - ready to build ChatbotV2!


## Load Metadata V2 and Training Set

In [ ]:
print("="*80)
print("📂 LOADING V2 FILES")
print("="*80)

# Load metadata_v2.json
print("\n📋 Loading metadata_v2.json...")
try:
    with open(METADATA_V2_PATH, 'r', encoding='utf-8') as f:
        metadata_v2 = json.load(f)
    
    print(f"✅ Metadata V2 loaded successfully!")
    print(f"   Tables: {len(metadata_v2['tables'])}")
    print(f"   Critical rules: {len(metadata_v2['critical_rules'])}")
    print(f"   Join patterns: {len(metadata_v2['join_patterns'])}")
    print(f"   Anti-patterns: {len(metadata_v2['anti_patterns'])}")
    print(f"   Reference data: {len(metadata_v2['reference_data'])} categories")
except FileNotFoundError:
    print(f"❌ Error: metadata_v2.json not found at {METADATA_V2_PATH}")
    metadata_v2 = None
except Exception as e:
    print(f"❌ Error loading metadata_v2.json: {e}")
    metadata_v2 = None

# Load training_set.json
print("\n📚 Loading training_set.json...")
try:
    with open(TRAINING_SET_PATH, 'r', encoding='utf-8') as f:
        training_data = json.load(f)
    
    training_examples = training_data['training_examples']
    
    print(f"✅ Training set loaded successfully!")
    print(f"   Total examples: {len(training_examples)}")
    
    # Count by category
    categories = {}
    for example in training_examples:
        cat = example['category']
        categories[cat] = categories.get(cat, 0) + 1
    
    print(f"\n   Examples by category:")
    for cat, count in sorted(categories.items(), key=lambda x: x[1], reverse=True):
        print(f"      - {cat}: {count}")
    
except FileNotFoundError:
    print(f"❌ Error: training_set.json not found at {TRAINING_SET_PATH}")
    training_examples = None
except Exception as e:
    print(f"❌ Error loading training_set.json: {e}")
    training_examples = None

print("\n" + "="*80)
if metadata_v2 and training_examples:
    print("✅ All V2 files loaded successfully!")
    print(f"📊 Ready for RAG Text-to-SQL with {len(training_examples)} training examples")
else:
    print("❌ Failed to load V2 files - check paths and file integrity")
print("="*80)

📂 LOADING V2 FILES

📋 Loading metadata_v2.json...
✅ Metadata V2 loaded successfully!
   Tables: 9
   Critical rules: 8
   Join patterns: 11
   Anti-patterns: 5
   Reference data: 6 categories

📚 Loading training_set.json...
✅ Training set loaded successfully!
   Total examples: 200

   Examples by category:
      - payment: 26
      - product: 26
      - customer: 25
      - time: 25
      - seller: 22
      - geographic: 20
      - multi_complex: 17
      - review: 13
      - general: 12
      - revenue: 10
      - edge_case: 4

✅ All V2 files loaded successfully!
📊 Ready for RAG Text-to-SQL with 200 training examples


### Initialize ChromaDB and Create Collection

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import os

print("="*80)
print("🔗 INITIALIZING LANGCHAIN COMPONENTS (ONE-TIME SETUP)")
print("="*80)

# ============================================================================
# STEP 1: Initialize Embeddings
# ============================================================================

print("\n📥 Initializing embeddings...")
langchain_embeddings = HuggingFaceEmbeddings(
    model_name='all-MiniLM-L6-v2',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ Embeddings initialized!")

# ============================================================================
# STEP 2: Prepare Examples
# ============================================================================

print("\n📋 Preparing training examples...")
langchain_examples = []
for example in training_examples:
    langchain_examples.append({
        "input": example['question'],  # Changed to "input" for LangChain compatibility!
        "sql": example['sql']
    })

print(f"✅ Prepared {len(langchain_examples)} examples")

# ============================================================================
# STEP 3: Create Example Selector with PERSISTENT ChromaDB
# ============================================================================

print("\n🔄 Creating vector store and embedding examples...")
print("   (This takes ~30-60 seconds, only happens once!)")

# Get ChromaDB path from environment
CHROMADB_PATH = os.getenv("CHROMADB_PATH", "./chroma_db")
COLLECTION_NAME = "training_examples"

print(f"   Saving to: {CHROMADB_PATH}")

# Create example selector with PERSISTENT storage
example_selector = SemanticSimilarityExampleSelector.from_examples(
    langchain_examples,
    langchain_embeddings,
    Chroma,
    k=7,
    persist_directory=CHROMADB_PATH,  # ← PERSIST TO DISK!
    collection_name=COLLECTION_NAME
)

print(f"✅ Vector store created and persisted to disk!")
print(f"   Location: {CHROMADB_PATH}")
print(f"   Collection: {COLLECTION_NAME}")
print(f"   Examples embedded: {len(langchain_examples)}")

# ============================================================================
# STEP 4: Define Example Template
# ============================================================================

print("\n📝 Defining example template...")

# Template for formatting individual examples
example_template = """Q: {input}
SQL: {sql}"""

example_prompt = PromptTemplate(
    input_variables=["input", "sql"],  # Changed "question" to "input"
    template=example_template
)

print("✅ Example template ready!")

print("\n" + "="*80)
print("✅ LangChain components ready!")
print("="*80)

🔗 INITIALIZING LANGCHAIN COMPONENTS (ONE-TIME SETUP)

📥 Initializing embeddings...


/var/folders/5v/rkkdx9h52qz3xmpjgnf6wrtw0000gn/T/ipykernel_68151/745430312.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  langchain_embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Embeddings initialized!

📋 Preparing training examples...
✅ Prepared 200 examples

🔄 Creating vector store and embedding examples...
   (This takes ~30-60 seconds, only happens once!)
   Saving to: ./chroma_db
✅ Vector store created and persisted to disk!
   Location: ./chroma_db
   Collection: training_examples
   Examples embedded: 200

📝 Defining example template...
✅ Example template ready!

✅ LangChain components ready!


### Build Prompt Template

In [ ]:

from langchain_core.prompts import FewShotPromptTemplate

print("Building prompt template...")

# Format static content from metadata
equals_line = "=" * 80

schema_text = ""
for table_name, columns in metadata_v2['table_columns_index'].items():
    schema_text += f"{table_name}: {columns}\n"

rules_text = ""
for i, rule in enumerate(metadata_v2['critical_rules'], 1):
    rules_text += f"{i}. {rule}\n"

antipatterns_text = ""
for pattern in metadata_v2['anti_patterns']:
    antipatterns_text += f"{pattern}\n"

# Create FewShotPromptTemplate with RAG
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # From Cell 11A
    example_prompt=example_prompt,      # From Cell 11A
    prefix=f"""DATABASE SCHEMA:
{equals_line}

{schema_text}

CRITICAL BUSINESS RULES:
{equals_line}

{rules_text}

ANTI-PATTERNS (NEVER DO THIS):
{equals_line}

{antipatterns_text}

SIMILAR EXAMPLES (Learn from these examples):
{equals_line}

""",
    suffix=f"""
USER QUESTION:
{equals_line}

Q: {{input}}

IMPORTANT: Output ONLY the SQL query. Do NOT include explanations, markdown, or any text before/after the SQL.
Just output the SQL query directly, nothing else.

SQL: """,
    input_variables=["input"],
)

def build_prompt_v2(question: str) -> str:
    """Build prompt with RAG-retrieved examples."""
    return few_shot_prompt.format(input=question)

print("✅ Prompt template ready\n")

Building prompt template...
✅ Prompt template ready



### Build SQL Validation Layer

In [ ]:
def validate_sql(sql: str) -> tuple[bool, str]:
    """
    Validate generated SQL before execution.
    
    Checks:
    1. Forbidden keywords (DROP, DELETE, INSERT, UPDATE, etc.)
    2. Must start with SELECT
    3. No SELECT * without aggregation
    
    Args:
        sql: Generated SQL query string
    
    Returns:
        (is_valid, message): Tuple of (bool, str)
            - True, "Valid" if SQL is safe
            - False, "Error message" if SQL is forbidden
    """
    
    sql_upper = sql.upper().strip()
    
    # ========================================================================
    # CHECK 1: Forbidden Keywords (from metadata_v2 guardrails)
    # ========================================================================
    
    forbidden_keywords = metadata_v2['guardrails']['forbidden_keywords']
    
    for keyword in forbidden_keywords:
        if keyword in sql_upper:
            return False, f"🚫 SECURITY ERROR: '{keyword}' operations are not allowed. This is a read-only system."
    
    # ========================================================================
    # CHECK 2: Must Start with SELECT
    # ========================================================================
    
    if not sql_upper.startswith('SELECT'):
        return False, "🚫 ERROR: Only SELECT queries are allowed. Query must start with SELECT."
    
    # ========================================================================
    # CHECK 3: No SELECT * (must use aggregations or specific columns)
    # ========================================================================
    
    if 'SELECT *' in sql_upper:
        # Allow SELECT * only if there's an aggregation in subquery
        if not any(agg in sql_upper for agg in ['COUNT(', 'SUM(', 'AVG(', 'MIN(', 'MAX(']):
            return False, "🚫 ERROR: SELECT * is forbidden. Please use specific columns or aggregations (COUNT, SUM, AVG, MIN, MAX)."
    
    # ========================================================================
    # All checks passed
    # ========================================================================
    
    return True, "✅ Valid"


# Test the validation layer
print("="*80)
print("🔒 TESTING SQL VALIDATION LAYER")
print("="*80)

test_queries = [
    # Valid queries
    ("SELECT COUNT(*) FROM customers", True),
    ("SELECT customer_id, customer_city FROM customers LIMIT 10", True),
    ("SELECT SUM(price) FROM order_items", True),
    
    # Invalid queries - Security threats
    ("DROP TABLE orders", False),
    ("DELETE FROM customers WHERE customer_id = '123'", False),
    ("TRUNCATE TABLE orders", False),
    ("INSERT INTO customers VALUES ('test')", False),
    ("UPDATE orders SET order_status = 'delivered'", False),
    
    # Invalid queries - SELECT * without aggregation
    ("SELECT * FROM orders", False),
    ("SELECT * FROM customers WHERE customer_state = 'SP'", False),
    
    # Valid - SELECT * with aggregation (subquery)
    ("SELECT * FROM (SELECT customer_id, COUNT(*) as cnt FROM orders GROUP BY customer_id) AS subq", True),
]

print("\n🧪 Testing validation with various SQL queries:\n")

passed = 0
failed = 0

for sql, should_pass in test_queries:
    is_valid, message = validate_sql(sql)
    
    # Check if result matches expectation
    if is_valid == should_pass:
        status = "✅ PASS"
        passed += 1
    else:
        status = "❌ FAIL"
        failed += 1
    
    print(f"{status}")
    print(f"   SQL: {sql[:80]}{'...' if len(sql) > 80 else ''}")
    print(f"   Expected: {'Valid' if should_pass else 'Invalid'}")
    print(f"   Result: {message}")
    print()

print("="*80)
print(f"📊 VALIDATION TEST RESULTS:")
print(f"   Passed: {passed}/{len(test_queries)}")
print(f"   Failed: {failed}/{len(test_queries)}")
print("="*80)

if failed == 0:
    print("✅ All validation tests passed! Security layer is working correctly.")
else:
    print(f"⚠️  {failed} test(s) failed. Review validation logic.")

print("="*80)

🔒 TESTING SQL VALIDATION LAYER

🧪 Testing validation with various SQL queries:

✅ PASS
   SQL: SELECT COUNT(*) FROM customers
   Expected: Valid
   Result: ✅ Valid

✅ PASS
   SQL: SELECT customer_id, customer_city FROM customers LIMIT 10
   Expected: Valid
   Result: ✅ Valid

✅ PASS
   SQL: SELECT SUM(price) FROM order_items
   Expected: Valid
   Result: ✅ Valid

✅ PASS
   SQL: DROP TABLE orders
   Expected: Invalid
   Result: 🚫 SECURITY ERROR: 'DROP' operations are not allowed. This is a read-only system.

✅ PASS
   SQL: DELETE FROM customers WHERE customer_id = '123'
   Expected: Invalid
   Result: 🚫 SECURITY ERROR: 'DELETE' operations are not allowed. This is a read-only system.

✅ PASS
   SQL: TRUNCATE TABLE orders
   Expected: Invalid
   Result: 🚫 SECURITY ERROR: 'TRUNCATE' operations are not allowed. This is a read-only system.

✅ PASS
   SQL: INSERT INTO customers VALUES ('test')
   Expected: Invalid
   Result: 🚫 SECURITY ERROR: 'INSERT' operations are not allowed. This is a rea

### ChatbotV2 with Natural Language Responses

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import re

print("Initializing ChatbotV2...")

# Initialize Ollama LLM
llm = Ollama(
    model=OLLAMA_MODEL,
    base_url=OLLAMA_BASE_URL,
    temperature=0.0,
)

# Initialize database connection
db_uri = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
db = SQLDatabase.from_uri(db_uri)

# SQL cleaning function
def clean_sql(sql: str) -> str:
    """Remove markdown formatting from LLM-generated SQL."""
    sql = re.sub(r'```sql\s*', '', sql)
    sql = re.sub(r'```\s*', '', sql)
    sql = sql.strip()
    sql = sql.rstrip(';')
    return sql

# Create SQL generation chain (RAG + LLM)
sql_generation_chain = (
    {"input": RunnablePassthrough()}
    | few_shot_prompt
    | llm
    | StrOutputParser()
    | clean_sql
)

# Natural language response generator
def generate_natural_response(question: str, sql: str, results: any) -> str:
    """Generate natural language response from SQL results."""
    
    response_prompt = f"""You are a helpful data analyst assistant. Answer the user's question in a natural, conversational way based on the query results.

USER'S QUESTION:
{question}

SQL QUERY EXECUTED:
{sql}

QUERY RESULTS:
{results}

INSTRUCTIONS:
- Answer the question directly in natural language
- Use the actual numbers/data from the results
- Be conversational and friendly
- Keep it concise (2-3 sentences max)
- Start with phrases like "According to our database..." or "Based on the data..." or "The analysis shows..."
- If results are empty, say "No data found for this query"
- Don't explain the SQL or how you got the answer, just answer the question

NATURAL LANGUAGE ANSWER:"""

    try:
        natural_response = llm.invoke(response_prompt)
        return natural_response.strip()
    except Exception as e:
        return f"Results: {results}"

# Complete pipeline function
def process_query(question: str, verbose: bool = True) -> dict:
    """
    Process natural language question and return natural language response.
    
    Pipeline: Question → RAG → SQL → Validate → Execute → Natural Response
    
    Args:
        question: User's question in natural language
        verbose: Print processing steps
        
    Returns:
        Dictionary with question, SQL, results, and natural response
    """
    
    if verbose:
        print("="*80)
        print(f"❓ QUESTION: {question}")
        print("="*80)
    
    try:
        # Generate SQL
        if verbose:
            print("\n🧠 Generating SQL...")
        
        sql = sql_generation_chain.invoke(question)
        
        if verbose:
            print(f"✅ Generated SQL:\n{sql}\n")
        
        # Validate SQL
        if verbose:
            print("🔒 Validating SQL...")
        
        is_valid, validation_message = validate_sql(sql)
        
        if verbose:
            print(f"{validation_message}\n")
        
        if not is_valid:
            return {
                "question": question,
                "sql": sql,
                "error": validation_message,
                "results": None,
                "response": f"I cannot execute this query due to security restrictions: {validation_message}"
            }
        
        # Execute SQL
        if verbose:
            print("⚡ Executing SQL...")
        
        results = db.run(sql)
        
        if verbose:
            print(f"✅ Query executed successfully!\n")
        
        # Generate natural language response
        if verbose:
            print("💬 Generating natural language response...")
        
        natural_response = generate_natural_response(question, sql, results)
        
        if verbose:
            print(f"✅ Response:\n   {natural_response}\n")
        
        return {
            "question": question,
            "sql": sql,
            "results": results,
            "response": natural_response,
            "success": True
        }
        
    except Exception as e:
        if verbose:
            print(f"\n❌ Error: {str(e)}\n")
        
        return {
            "question": question,
            "sql": sql if 'sql' in locals() else None,
            "error": str(e),
            "results": None,
            "response": f"I encountered an error while processing your question: {str(e)}"
        }

print("✅ ChatbotV2 ready\n")

Initializing ChatbotV2...
✅ ChatbotV2 ready



### Comprehensive Testing - 15 Questions (WITH NATURAL RESPONSES)

In [ ]:
import pandas as pd
from datetime import datetime

print("="*80)
print("🧪 COMPREHENSIVE TESTING - 15 QUESTIONS (NATURAL RESPONSES)")
print("="*80)
print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# Test questions - Simple list (LLM only sees the strings!)
test_questions = [
    # Easy (In training set)
    "How many unique customers do we have?",
    "What's the total revenue?",
    "How many orders were placed?",
    
    # Medium (Not in training)
    "How many customers are from Rio de Janeiro city?",
    "What's the average order value?",
    "Show me the top 3 product categories by sales",
    
    # Hard (Complex)
    "How many customers have spent more than R$1000 total?",
    "What's the revenue by payment type?",
    "Which state has the highest number of orders?",
    
    # Edge cases (Business logic)
    "How many products are in the Electronics category?",
    "What's the revenue in January 2018?",
    "How many repeat customers are there?",
    
    # Security (Should be blocked)
    "TRUNCATE TABLE orders",
    "Delete all customers from São Paulo",
    
    # Invalid/Ambiguous
    "Show me all information about everything",
]

# Store results
results = []

# Process each question
for i, question in enumerate(test_questions, 1):
    print("="*80)
    print(f"TEST {i}/{len(test_questions)}")
    print("="*80)
    print(f"❓ {question}")
    print("─"*80)
    
    try:
        import time
        start = time.time()
        
        # Process query
        result = process_query(question, verbose=False)
        
        elapsed = time.time() - start
        
        # Display results
        if result.get('error'):
            print(f"🚫 BLOCKED/ERROR:")
            print(f"   {result.get('response', result['error'])}")  # ← Show natural response!
            if result.get('sql'):
                print(f"   SQL: {result['sql'][:100]}")
            status = "BLOCKED" if any(word in result['error'] for word in ["BLOCKED", "forbidden", "not allowed"]) else "ERROR"
            response_preview = result.get('response', result['error'])[:80]
        elif result.get('success'):
            print(f"✅ SQL: {result['sql'][:100]}...")
            print(f"💬 RESPONSE: {result.get('response', 'No response')}")  # ← Show natural response!
            status = "SUCCESS"
            response_preview = result.get('response', 'No response')[:80]
        else:
            print(f"❌ FAILED: {result}")
            status = "FAILED"
            response_preview = "Failed"
        
        print(f"⏱️  {elapsed:.2f}s")
        
        # Store result
        results.append({
            'ID': i,
            'Question': question[:60] + '...' if len(question) > 60 else question,
            'Status': status,
            'Response': response_preview,  # ← Store natural response!
            'SQL': result.get('sql', 'N/A')[:80] + '...' if result.get('sql') and len(result.get('sql', '')) > 80 else result.get('sql', 'N/A'),
            'Time(s)': f"{elapsed:.2f}"
        })
        
    except Exception as e:
        print(f"❌ EXCEPTION: {e}")
        results.append({
            'ID': i,
            'Question': question[:60],
            'Status': 'EXCEPTION',
            'Response': str(e)[:80],
            'SQL': 'N/A',
            'Time(s)': 'N/A'
        })
    
    print()

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*80)
print("📊 TEST RESULTS SUMMARY")
print("="*80)

# Create DataFrame
df = pd.DataFrame(results)

# Display table
print("\n📋 DETAILED RESULTS:")
print(df.to_string(index=False))

# Statistics
print("\n" + "="*80)
print("📈 STATISTICS")
print("="*80)

status_counts = df['Status'].value_counts()
total = len(results)

print(f"\nTotal Tests: {total}")
print(f"\nBreakdown:")
for status, count in status_counts.items():
    percentage = (count / total) * 100
    print(f"   {status:12s}: {count:2d} ({percentage:5.1f}%)")

success_count = status_counts.get('SUCCESS', 0)
blocked_count = status_counts.get('BLOCKED', 0)

print(f"\n✅ Success Rate: {(success_count/total)*100:.1f}%")
print(f"🚫 Security Blocking: {(blocked_count/total)*100:.1f}%")

print("\n" + "="*80)
print(f"✅ Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

🧪 COMPREHENSIVE TESTING - 15 QUESTIONS (NATURAL RESPONSES)
Started at: 2026-02-11 00:30:40

TEST 1/15
❓ How many unique customers do we have?
────────────────────────────────────────────────────────────────────────────────
✅ SQL: SELECT COUNT(DISTINCT customer_unique_id) AS unique_customers FROM customers...
💬 RESPONSE: According to our database, we have 96,096 unique customers.
⏱️  32.46s

TEST 2/15
❓ What's the total revenue?
────────────────────────────────────────────────────────────────────────────────
✅ SQL: SELECT SUM(oi.price) AS total_revenue 
FROM order_items oi 
JOIN orders o ON oi.order_id = o.order_i...
💬 RESPONSE: According to our database, the total revenue from delivered orders is $13,221,498.11.
⏱️  22.68s

TEST 3/15
❓ How many orders were placed?
────────────────────────────────────────────────────────────────────────────────
✅ SQL: SELECT COUNT(DISTINCT order_id) AS total_orders FROM orders...
💬 RESPONSE: According to our database, 99,441 orders were placed.
⏱️  17.0

### Advanced Testing - Complex Questions (WITH NATURAL RESPONSES)

In [ ]:
print("="*80)
print("🔬 ADVANCED TESTING - 15 COMPLEX QUESTIONS (NATURAL RESPONSES)")
print("="*80)
print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# Advanced test questions
test_questions = [
    # Temporal & Trend Analysis
    "What was the month-over-month revenue growth in 2017?",
    "Which day of the week has the highest order volume?",
    "Show me customers who made their first purchase in 2017 but haven't ordered since",
    
    # Multi-Dimensional Aggregations
    "What's the average delivery time by state for orders over R$500?",
    "Which product category has the highest customer retention rate?",
    "Compare the average review score between credit card and boleto payments",
    
    # Percentile & Ranking
    "Show me the top 5 sellers by revenue, but exclude sellers with less than 100 orders",
    "What percentage of total revenue comes from the top 10% of customers?",
    "Find customers in the 90th percentile of spending",
    
    # Correlation & Patterns
    "Is there a correlation between product price and review score?",
    "Which states have above-average spending per customer?",
    "Show me product categories where average delivery time exceeds 30 days",
    
    # Edge Cases & Anomalies
    "Find orders with more than 10 items",
    "Which customers have made purchases in more than 3 different states?",
    
    # Tricky Business Logic
    "What's the cancellation rate by product category?",
]

# Store results
results = []

# Process each question
for i, question in enumerate(test_questions, 1):
    print("="*80)
    print(f"TEST {i}/{len(test_questions)}")
    print("="*80)
    print(f"❓ {question}")
    print("─"*80)
    
    try:
        import time
        start = time.time()
        
        # Process query
        result = process_query(question, verbose=False)
        
        elapsed = time.time() - start
        
        # Display results
        if result.get('error'):
            print(f"🚫 ERROR:")
            print(f"   💬 {result.get('response', result['error'])}")  # ← Natural response
            if result.get('sql'):
                print(f"   SQL: {result['sql'][:120]}...")
            status = "ERROR"
            response_preview = result.get('response', result['error'])[:100]
        elif result.get('success'):
            print(f"✅ SQL: {result['sql'][:120]}...")
            print(f"💬 RESPONSE:")
            print(f"   {result.get('response', 'No response')}")  # ← Natural response!
            status = "SUCCESS"
            response_preview = result.get('response', 'No response')[:100]
        else:
            print(f"❌ FAILED: {result}")
            status = "FAILED"
            response_preview = "Failed"
        
        print(f"⏱️  {elapsed:.2f}s")
        
        # Store result
        results.append({
            'ID': i,
            'Question': question[:70] + '...' if len(question) > 70 else question,
            'Status': status,
            'Response': response_preview,  # ← Natural response column!
            'Time(s)': f"{elapsed:.2f}"
        })
        
    except Exception as e:
        print(f"❌ EXCEPTION: {e}")
        results.append({
            'ID': i,
            'Question': question[:70],
            'Status': 'EXCEPTION',
            'Response': str(e)[:100],
            'Time(s)': 'N/A'
        })
    
    print()

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*80)
print("📊 ADVANCED TEST RESULTS SUMMARY")
print("="*80)

# Create DataFrame
df = pd.DataFrame(results)

# Display table
print("\n" + df.to_string(index=False))

# Statistics
print("\n" + "="*80)
print("📈 STATISTICS")
print("="*80)

status_counts = df['Status'].value_counts()
total = len(results)

print(f"\nTotal Tests: {total}")
print(f"\nBreakdown:")
for status, count in status_counts.items():
    percentage = (count / total) * 100
    print(f"   {status:12s}: {count:2d} ({percentage:5.1f}%)")

success_count = status_counts.get('SUCCESS', 0)

print(f"\n🎯 Advanced Query Success Rate: {(success_count/total)*100:.1f}%")

if success_count > 10:
    print("\n🏆 EXCELLENT - System handles complex queries with natural responses!")
elif success_count > 7:
    print("\n✅ GOOD - Most complex queries answered naturally!")
else:
    print("\n⚠️  FAIR - Some improvement needed")

print("\n" + "="*80)
print(f"✅ Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

🔬 ADVANCED TESTING - 15 COMPLEX QUESTIONS (NATURAL RESPONSES)
Started at: 2026-02-11 00:38:59

TEST 1/15
❓ What was the month-over-month revenue growth in 2017?
────────────────────────────────────────────────────────────────────────────────
✅ SQL: SELECT 
    EXTRACT(MONTH FROM o.order_purchase_timestamp) AS month,
    SUM(oi.price) AS revenue,
    LAG(SUM(oi.price)...
💬 RESPONSE:
   According to our database, the month-over-month revenue growth in 2017 varied significantly throughout the year. For instance, there was a notable increase of 53.4% from February to March, while there was also a decline of 13.8% from May to June.
⏱️  46.15s

TEST 2/15
❓ Which day of the week has the highest order volume?
────────────────────────────────────────────────────────────────────────────────
✅ SQL: SELECT EXTRACT(DOW FROM order_purchase_timestamp) AS day_of_week, COUNT(DISTINCT order_id) AS order_count 
FROM orders 
...
💬 RESPONSE:
   According to our database, Monday (represented by day of week 